# Notebook 03 — Modeling & Export

**Goal:** Train baseline (LogReg, RF), tune RF, evaluate, and export the best pipeline.  
**Input:** `../data/processed/hr_attrition_ready.parquet`  
**Outputs:** `../artifacts/v1/rf_pipeline.joblib`, `../artifacts/v1/features.json`  
